<a href="https://colab.research.google.com/github/Sriramsai-Mutyala/multilingual-ner/blob/main/sriramsai7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
np.random.seed(0)

In [ ]:
import pandas as pd
import zipfile

# Specify the path to the ZIP file
zip_file_path = '/content/archive (1) (1).zip'
# Specify the name of the CSV file you want to read
csv_file_name = 'kannada_ner.csv'

# Open the ZIP file and read the specified CSV file
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    with zip_file.open(csv_file_name) as csv_file:
        data1 = pd.read_csv(csv_file, encoding='UTF-8')

# Display the first 10 rows of the DataFrame
data1.head(10)

,tokens,tags
0,ಕನ್ನಡ,NNP
1,ರಂಗಭೂಮಿ,NN
2,",",SYM
3,ಚಿತ್ರರಂಗಗಳಲ್ಲದೆ,UNK
4,ಇತರ,JJ
5,ಭಾಷೆಗಳಲ್ಲೂ,NN
6,ಪ್ರಸಿದ್ಧಿ,NN
7,ಪಡೆದವರು,VM
8,.,SYM
9,ಇವರು,PRP


In [ ]:
data1.isnull().sum()


tokens    0
tags      0
dtype: int64

In [ ]:
print("Unique Words in corpus:",data1['tokens'].nunique())
print("Unique Tag in corpus:",data1['tags'].nunique())

Unique Words in corpus: 34233
Unique Tag in corpus: 24


In [ ]:
print(data1.columns)

Index(['tokens', 'tags'], dtype='object')


In [ ]:
words = list(set(data1['tokens'].values))
words.append("ENDPAD")
num_words = len(words)
tags = list(set(data1['tags'].values))
num_tags = len(tags)
num_words,num_tags

(34234, 24)

In [ ]:
import pandas as pd

# Assuming 'Tag' is the column containing tags in your DataFrame
tags = list(data1['tags'].values)
tags.append("ENDPAD")

num_tags = data1['tags'].nunique()  # Number of unique tags excluding 'ENDPAD'
tag_values = data1['tags'].unique()  # Unique tag values excluding 'ENDPAD'

# Filter out 'ENDPAD' if it exists
tag_values = [tag for tag in tag_values if tag != 'ENDPAD']

print("Number of Tags:", num_tags)
print("Tag Values (excluding 'ENDPAD'):", tag_values)

Number of Tags: 24
Tag Values (excluding 'ENDPAD'): ['NNP', 'NN', 'SYM', 'UNK', 'JJ', 'VM', 'PRP', 'QC', 'CC', 'DEM', 'RB', 'QF', 'NST', 'PSP', 'INTF', 'RP', 'QO', 'WQ', 'NEG', 'CL', 'RDP', 'UT', 'ECH', 'INJ']


In [ ]:
train_data = []

for index, row in data1.iterrows():
    text = row['tokens']

    # Check if 'tags' column is NaN, and skip the row if it is
    if pd.isna(row['tags']):
        continue

    entities = {'entities': []}
    tags = row['tags']

    start = None
    for i, tag in enumerate(tags):
        if tag.startswith('B-'):
            if start is not None:
                entities['entities'].append((start, i, tags[i-1][2:]))
            start = i
        elif tag.startswith('I-'):
            continue
        else:  # O tag or other
            if start is not None:
                entities['entities'].append((start, i, tags[i-1][2:]))
            start = None

    train_data.append((text, entities))


In [ ]:
import spacy
from spacy.training.example import Example

# Assuming you have already loaded the spaCy model
nlp = spacy.load("en_core_web_sm")

# Convert train_data to spaCy training examples
training_examples = []
for text, entities in train_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, entities)
    training_examples.append(example)

# Update the NER model with the new training data
ner = nlp.get_pipe("ner")
ner.update(training_examples, drop=0.5, losses={})

# Save the updated model if needed
nlp.to_disk("/content/output")


In [ ]:
# Load the updated model
updated_nlp = spacy.load("/content/output")

# Example text for prediction
new_text = "ಹೊಸ ವರ್ಷದಲ್ಲಿ ನೀವು ಎಷ್ಟು ಚಿಕ್ಕದೊ ಅಥವಾ ದೊಡ್ಡದೊ ಆಗಲಿ, ಅದು ನಿಮ್ಮ ಪ್ರಯಾಸಗಳನ್ನು ಹೆಚ್ಚಿಸಲಿ ಮತ್ತು ಸಫಲವಾಗಿ ಹೋಗಲಿ. ನೂತನ ವರ್ಷದ ಶುಭಾಶಯಗಳು"

# Process the text with the updated model
doc = updated_nlp(new_text)

# Access entities in the processed document
for ent in doc.ents:
    print(f"Entity: {ent.text}, Type: {ent.label_}")


Entity: ಚಿಕ್ಕದೊ ಅಥವಾ, Type: PERSON


In [ ]:
import pandas as pd

# Assuming 'Tag' is the column containing tags in your DataFrame
tags = list(data1['tags'].values)
tags.append("ENDPAD")

# Count the occurrences of each tag
tag_counts = pd.Series(tags).value_counts()

# Remove the count for 'ENDPAD' if it exists
tag_counts = tag_counts.drop('ENDPAD', errors='ignore')

# Print the total number of tags
total_tags = tag_counts.sum()
print("Total Number of Tags:", total_tags)

# Print the tag counts
print("Tag Counts:")
print(tag_counts)

Total Number of Tags: 100479
Tag Counts:
NN      34408
UNK     14405
SYM     13625
VM      10378
NNP      6614
JJ       4596
PRP      4477
QC       2884
CC       2731
RB       1673
NST      1632
DEM      1246
QF        721
INTF      420
QO        314
RP        222
WQ         54
NEG        31
PSP        26
CL         10
RDP         4
ECH         4
INJ         3
UT          1
dtype: int64


In [ ]:
!python -m spacy download xx_ent_wiki_sm


2024-01-05 13:53:49.960122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 13:53:49.960168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 13:53:49.960966: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 13:53:50.839560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 38.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')


In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!python -m spacy download en_core_web_sm

2024-01-05 13:54:08.150968: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 13:54:08.151019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 13:54:08.152044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_

In [ ]:
import spacy
# Load the small multilingual spaCy model
nlp = spacy.load("xx_ent_wiki_sm")

In [ ]:
# Specify the name of the new directory
new_directory_name = 'output'
# Create the new directory
!mkdir $new_directory_name


mkdir: cannot create directory ‘output’: File exists


In [ ]:
spacy.prefer_gpu()

False

In [ ]:
# Import necessary libraries
import spacy

# Load the spaCy model (replace 'en_core_web_sm' with the appropriate model)
nlp = spacy.load('en_core_web_sm')

# Define the number of training iterations
num_iterations = 100  # Adjust this number as needed

# Get the ner component of the pipeline
ner = nlp.get_pipe("ner")

# Prepare and format training data (replace train_data with your actual training data)
train_data = [
    ("Your text here", {"entities": [(0, 4, "LABEL")]}),
    # Add more examples as needed
]

# Train the model
# Train the model
for iteration in range(num_iterations):
    # Iterate over the training data
    for text, annotations in train_data:
        # Update the Named Entity Recognition model
        example = spacy.training.example.Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], drop=0.5)

    # Print some information after each iteration
    print(f"Iteration {iteration + 1} completed.")

# Save the trained model to a file
nlp.to_disk("output_model")
print("Training completed. Model saved.")



Iteration 1 completed.
Iteration 2 completed.
Iteration 3 completed.
Iteration 4 completed.
Iteration 5 completed.
Iteration 6 completed.
Iteration 7 completed.
Iteration 8 completed.
Iteration 9 completed.
Iteration 10 completed.
Iteration 11 completed.
Iteration 12 completed.
Iteration 13 completed.
Iteration 14 completed.
Iteration 15 completed.
Iteration 16 completed.
Iteration 17 completed.
Iteration 18 completed.
Iteration 19 completed.
Iteration 20 completed.
Iteration 21 completed.
Iteration 22 completed.
Iteration 23 completed.
Iteration 24 completed.
Iteration 25 completed.
Iteration 26 completed.
Iteration 27 completed.
Iteration 28 completed.
Iteration 29 completed.
Iteration 30 completed.
Iteration 31 completed.
Iteration 32 completed.
Iteration 33 completed.
Iteration 34 completed.
Iteration 35 completed.
Iteration 36 completed.
Iteration 37 completed.
Iteration 38 completed.
Iteration 39 completed.
Iteration 40 completed.
Iteration 41 completed.
Iteration 42 completed.
I

In [ ]:
# Load the trained model from disk
nlp = spacy.load("output_model")

# Test the trained model on new text
test_text = "Your test text here"
doc = nlp(test_text)

# Print recognized entities and their labels
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")


Entity: Your, Label: LABEL


In [ ]:
text = "ಹೊಸ ವರ್ಷದಲ್ಲಿ ನೀವು ಎಷ್ಟು ಚಿಕ್ಕದೊ ಅಥವಾ ದೊಡ್ಡದೊ ಆಗಲಿ, ಅದು ನಿಮ್ಮ ಪ್ರಯಾಸಗಳನ್ನು ಹೆಚ್ಚಿಸಲಿ ಮತ್ತು ಸಫಲವಾಗಿ ಹೋಗಲಿ. ನೂತನ ವರ್ಷದ ಶುಭಾಶಯಗಳು"
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

ಚಿಕ್ಕದೊ ಅಥವಾ PERSON


In [ ]:
!pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 10.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=2600b174792798a6484bae5b29ec2b2c124e2f3211833434e9fbe76e6a9cf7c8
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling

In [ ]:
from googletrans import Translator

def translate_hindi_to_english(hindi_text):
    # Create a Translator object
    translator = Translator()

    # Translate Hindi text to English
    translation = translator.translate(hindi_text, src='hi', dest='en')

    # Return the translated text
    return translation.text

# Example Hindi text
hindi_text = "ಹೊಸ ವರ್ಷದಲ್ಲಿ ನೀವು ಎಷ್ಟು ಚಿಕ್ಕದೊ ಅಥವಾ ದೊಡ್ಡದೊ ಆಗಲಿ, ಅದು ನಿಮ್ಮ ಪ್ರಯಾಸಗಳನ್ನು ಹೆಚ್ಚಿಸಲಿ ಮತ್ತು ಸಫಲವಾಗಿ ಹೋಗಲಿ. ನೂತನ ವರ್ಷದ ಶುಭಾಶಯಗಳು"

# Translate Hindi to English
english_translation = translate_hindi_to_english(hindi_text)

# Print the results
print("Hindi:", hindi_text)
print("English Translation:", english_translation)


Hindi: ಹೊಸ ವರ್ಷದಲ್ಲಿ ನೀವು ಎಷ್ಟು ಚಿಕ್ಕದೊ ಅಥವಾ ದೊಡ್ಡದೊ ಆಗಲಿ, ಅದು ನಿಮ್ಮ ಪ್ರಯಾಸಗಳನ್ನು ಹೆಚ್ಚಿಸಲಿ ಮತ್ತು ಸಫಲವಾಗಿ ಹೋಗಲಿ. ನೂತನ ವರ್ಷದ ಶುಭಾಶಯಗಳು
English Translation: ಹೊ ವರ ವರ ನೀವು ನೀವು ಎಷ ಎಷಟು ಚಿಅಥವ ಅಥವಾ ದೊಡದೊಡ ಆಗಲಿ, ಅದು ನಿಮ ನಿಮ ನಿಮನಿಮ ಪ್ನು ಗಳನ್ನು ಹೆಚಹೆಚ ಮತಮತ ಫಲವ ಹೋಗಲಿ ಹೋಗಲಿ.ನೂತನ ವರ್ಷದ ಶುಭಾಶಯಗಳು


In [ ]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [ ]:
raw_text=english_translation
text1= NER(raw_text)

In [ ]:
for word in text1.ents:
    print(word.text,word.label_)

In [ ]:
import spacy

def custom_entity_recognition(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    # Print detailed information for each token
    for token in doc:
        print(f"Word: {token.text}, POS: {token.pos_}")

# Example sentence
sentence = english_translation

# Perform custom entity recognition
custom_entity_recognition(sentence)


Word: ಹೊ, POS: PROPN
Word: ವರ, POS: PROPN
Word: ವರ, POS: NOUN
Word: ನೀವು, POS: PROPN
Word: ನೀವು, POS: PROPN
Word: ಎಷ, POS: PROPN
Word: ಎಷಟು, POS: PROPN
Word: ಚಿಅಥವ, POS: PUNCT
Word: ಅಥವಾ, POS: PROPN
Word: ದೊಡದೊಡ, POS: PROPN
Word: ಆಗಲಿ, POS: PROPN
Word: ,, POS: PUNCT
Word: ಅದು, POS: PROPN
Word: ನಿಮ, POS: NOUN
Word: ನಿಮ, POS: PUNCT
Word: ನಿಮನಿಮ, POS: PUNCT
Word: ಪ್ನು, POS: PUNCT
Word: ಗಳನ್ನು, POS: PUNCT
Word: ಹೆಚಹೆಚ, POS: PROPN
Word: ಮತಮತ, POS: PROPN
Word: ಫಲವ, POS: AUX
Word: ಹೋಗಲಿ, POS: PROPN
Word: ಹೋಗಲಿ.ನೂತನ, POS: NOUN
Word: ವರ್ಷದ, POS: PROPN
Word: ಶುಭಾಶಯಗಳು, POS: PROPN


In [ ]:
import spacy

# Load your trained spaCy NER model (replace "path/to/your/ner_model" with the actual path)
ner_model = spacy.load("/content/output")

# Save the trained model
ner_model.to_disk("/content/output")


In [ ]:
import spacy

# Load the trained spaCy NER model
ner_model = spacy.load("/content/output")

# Example text (replace with your dataset)
text = "ಹೊಸ ವರ್ಷದಲ್ಲಿ ನೀವು ಎಷ್ಟು ಚಿಕ್ಕದೊ ಅಥವಾ ದೊಡ್ಡದೊ ಆಗಲಿ, ಅದು ನಿಮ್ಮ ಪ್ರಯಾಸಗಳನ್ನು ಹೆಚ್ಚಿಸಲಿ ಮತ್ತು ಸಫಲವಾಗಿ ಹೋಗಲಿ. ನೂತನ ವರ್ಷದ ಶುಭಾಶಯಗಳು"

# Process the entire text with the loaded model
doc = ner_model(text)

# Print entities and their labels for each token (word)
for token in doc:
    print(f"Word: {token.text}, Entity: {doc.ents[token.i].text if token.i < len(doc.ents) else 'None'}, Label: {doc.ents[token.i].label_ if token.i < len(doc.ents) else 'None'}")


Word: ಹೊಸ, Entity: ಚಿಕ್ಕದೊ ಅಥವಾ, Label: PERSON
Word: ವರ್ಷದಲ್ಲಿ, Entity: None, Label: None
Word: ನೀವು, Entity: None, Label: None
Word: ಎಷ್ಟು, Entity: None, Label: None
Word: ಚಿಕ್ಕದೊ, Entity: None, Label: None
Word: ಅಥವಾ, Entity: None, Label: None
Word: ದೊಡ್ಡದೊ, Entity: None, Label: None
Word: ಆಗಲಿ, Entity: None, Label: None
Word: ,, Entity: None, Label: None
Word: ಅದು, Entity: None, Label: None
Word: ನಿಮ್ಮ, Entity: None, Label: None
Word: ಪ್ರಯಾಸಗಳನ್ನು, Entity: None, Label: None
Word: ಹೆಚ್ಚಿಸಲಿ, Entity: None, Label: None
Word: ಮತ್ತು, Entity: None, Label: None
Word: ಸಫಲವಾಗಿ, Entity: None, Label: None
Word: ಹೋಗಲಿ, Entity: None, Label: None
Word: ., Entity: None, Label: None
Word: ನೂತನ, Entity: None, Label: None
Word: ವರ್ಷದ, Entity: None, Label: None
Word: ಶುಭಾಶಯಗಳು, Entity: None, Label: None
